# Natural Language Systems (COMP34112) 
# Alexander Angelov<br>
## Coursework 1 (Task 1)

Importing the necessary libraries and downloading the corpuses(if needed).

In [1]:
from nltk import *
#download('brown')
#download('averaged_perceptron_tagger')
#download('punkt')

Functions for:
- Getting all occurences of a given tag in the given corpus.
- Getting the probability of a tag A following tag B in a given corpus.

In [2]:
def get_occurrences_of_tag(tag,corpora):
	return corpora[tag].N()
	
def get_prob_oftag_sequence(tag_1,tag_2,pos_tags,corpora):
	true_tag_pairs = 0
	brown_tag_pairs = bigrams(pos_tags)
	for (pair_1, pair_2) in brown_tag_pairs:
		if(pair_1[1] == tag_2 and pair_2[1] == tag_1):
			true_tag_pairs += 1
	return true_tag_pairs / get_occurrences_of_tag(tag_2,corpora)

Connecting each word with its POS tag in a tuple.

In [3]:
#sentence = word_tokenize("People continue to inquire the reason for the race for outer space")

brown_pos_tags = corpus.brown.tagged_words()
print(brown_pos_tags)

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]


Reversing the elements of each pair in order to get the number of occurences for each POS tag easily.

In [4]:
brown_cond_dist = ConditionalFreqDist(brown_pos_tags)
reversed_brown_cond_dist = ConditionalFreqDist((p_tag, word) for (word, p_tag) in brown_pos_tags)

Finding the necessary word likelihhod probabilities

In [5]:
################################################## Word likelihood probabilities

### P(race|NN) = C(NN,race) / C(NN)

c_NN_race = brown_cond_dist['race']['NN']
c_NN = get_occurrences_of_tag('NN',reversed_brown_cond_dist)
p_race_NN = c_NN_race / c_NN

### P(race|VB) = C(VB,race) / C(VB)

c_VB_race = brown_cond_dist['race']['VB']
c_VB = get_occurrences_of_tag('VB',reversed_brown_cond_dist)
p_race_VB = c_VB_race / c_VB

print("The word likelihood probabilities are: \n")
print("   P(race|NN) => " + str(p_race_NN) + "\n")
print("   P(race|VB) => " + str(p_race_VB) + "\n")

The word likelihood probabilities are: 

   P(race|NN) => 0.000616514724208041

   P(race|VB) => 0.00011871902175526073



Finding the necessary transmition probabilities and calculating the product needed for disambiguating which of the two given tags is more likely to be used.

In [6]:
################################################## Tag transition probabilities

### P(NN|AT) = (all AT NN sequences) / C(AT)

p_NN_AT = get_prob_oftag_sequence('NN','AT',brown_pos_tags,reversed_brown_cond_dist)
#print(list(word_tag_pairs))

### P(IN|NN) = (all NN IN sequences) / C(NN)

p_IN_NN = get_prob_oftag_sequence('IN','NN',brown_pos_tags,reversed_brown_cond_dist)

### P(VB|AT) = (all AT VB sequences) / C(AT)

p_VB_AT = get_prob_oftag_sequence('VB','AT',brown_pos_tags,reversed_brown_cond_dist)

### P(IN|VB) = (all NN IN sequences) / C(VB)

p_IN_VB = get_prob_oftag_sequence('IN','VB',brown_pos_tags,reversed_brown_cond_dist)

print("The tag transition probabilities are: \n")
print("   P(NN|AT) => " + str(p_NN_AT) + "\n")
print("   P(IN|NN) => " + str(p_IN_NN) + "\n")
print("   P(VB|AT) => " + str(p_VB_AT) + "\n")
print("   P(IN|VB) => " + str(p_IN_VB) + "\n")

print("Disambiguation: \n")
print("   P(race|NN)*P(NN|AT)*P(IN|NN) => " + str(p_race_NN * p_NN_AT * p_IN_NN) + "\n")
print("   P(race|VB)*P(VB|AT)*P(IN|VB) => " + str(p_race_VB * p_VB_AT * p_IN_VB) + "\n")

The tag transition probabilities are: 

   P(NN|AT) => 0.4938392592819445

   P(IN|NN) => 0.2771430445333508

   P(VB|AT) => 0.00016333363958390755

   P(IN|VB) => 0.14077108004630043

Disambiguation: 

   P(race|NN)*P(NN|AT)*P(IN|NN) => 8.437874262336417e-05

   P(race|VB)*P(VB|AT)*P(IN|VB) => 2.7296652541619724e-09



Calculating how many times POS tag NN is more likely to be used than POS tag VB.

In [7]:
print("POS tag NN is " + str((p_race_NN * p_NN_AT * p_IN_NN) / (p_race_VB * p_VB_AT * p_IN_VB)) + " times more likely to be used in this sentence.")

POS tag NN is 30911.754653692537 times more likely to be used in this sentence.
